# Oct 3, 2024: compare modewise communities
compare community realizations across modes to find 
1. if same rois have varying memberships in different modes, 
2. if rois themselves change within a community across modes: [all lower memberships in soft-comm],
3. is there a core set of rois for a community consistent across modes: [higher memberships in soft-comm],

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'positive'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'ses'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels = parcels_img.numpy()
roi_labels = np.loadtxt(f'{PARCELS_path}/{PARC_DESC}_desc-labels.txt')

In [6]:
def mem_entropy(args, pi):
    # membership entropy per marginal
    ent = stats.entropy(pi, base=2, axis=-1) / np.log2(pi.shape[-1])
    ent = np.round(np.nan_to_num(ent), decimals=3)
    rank_ent = np.round(stats.rankdata(ent) / pi.shape[0], decimals=3)
    return ent, rank_ent

In [7]:
def plot_entropy_comparison(args, ax, X, Y, level=-1):
    # mode entropies
    sns.stripplot(
        X, 
        ax=ax, 
        orient='h', 
        jitter=False, 
        size=4, 
        color='royalblue',
        alpha=0.4,
    )
    sns.boxplot(
        X, 
        ax=ax, 
        orient='h', 
        fill=False, 
        width=0.7, 
        fliersize=0.0, 
        color='mediumseagreen', 
        medianprops={"color": "mediumseagreen", "linewidth": 2}, 
        whis=[2.5, 97.5],
    )
    # soft entropy
    sns.stripplot(
        Y, 
        ax=ax,
        orient='h',
        jitter=False,
        color='lightcoral',
        # facecolor='lightcoral',
        # edgecolor='lightcoral',
        size=8,
        # linewidth=1,
    )
    ymin, ymax = ax.get_ylim()
    ax.axvline(x=0.25, ymin=ymin, ymax=ymax, color='lightsteelblue', linestyle='--', linewidth=2)
    ax.axvline(x=0.50, ymin=ymin, ymax=ymax, color='lightsteelblue', linestyle='--', linewidth=2)
    ax.axvline(x=0.75, ymin=ymin, ymax=ymax, color='lightsteelblue', linestyle='--', linewidth=2)

    if level == -1:
        title = '\n'.join([f'sub-{args.sub}_{SBM}', f'{args.stats_column}'])
    else:
        title = '\n'.join([f'sub-{args.sub}_{SBM}_level-{level}', f'{args.stats_column}'])
    ax.set(
        xlabel='entropy', 
        ylabel='roi', 
        title=title,
        xlim=[0, 1],
    )
    return ax

In [8]:
def get_mem_variability_type(args, X, Y):
    '''
    X: mode_ents; num_modes x num_rois
    Y: soft_ent; 1 x num_rois
    omegas: probabilities of mode_ents; num_modes x 1
    '''
    # X = np.repeat(X, (omegas*1000).astype(int), axis=0)
    cil = np.percentile(X, 25, axis=0)
    ciu = np.percentile(X, 75, axis=0)
    lims = np.stack(list(zip(cil, np.median(X, axis=0), ciu)), axis=0)
    within_mode_var = ((lims[:, 0] >= 0.02) | (lims[:, 2] >= 0.13)).astype(int)

    cil = np.percentile(X, 2.5, axis=0)
    ciu = np.percentile(X, 97.5, axis=0)
    across_mode_var = 1 - ((cil <= Y) & (Y <= ciu)).astype(int)

    var_types = np.stack(list(zip(within_mode_var, across_mode_var)), axis=0)
    var_types = np.array([int(''.join(map(str,var_type)), base=2) for var_type in var_types]).reshape(-1, 1)
    var_types = OneHotEncoder(sparse_output=False, categories=[np.array([0, 1, 2, 3])]).fit_transform(var_types)
    # var_types: matrix of size (num_rois x 4)
    return var_types

In [9]:
def get_mem_variability_type_per_roi(args, mode_ents, omegas, soft_ent, bandwidth_factor=3.5, num_points=1000):
    # data and gaussian kernel pdf
    data = np.repeat(mode_ents, (omegas*1000).astype(int))
    scott_factor = np.power(len(data), -1./(data.ndim + 4))
    kernel = stats.gaussian_kde(data, bw_method=scott_factor * bandwidth_factor)

    # evaluate kernel
    x_range = np.linspace(-0.1, 1.0, num_points)
    pdf = kernel(x_range)
    M = np.max(pdf) 
    pdf /= M # rescaling density to have max as 1.0

    # does soft_ent fall inside pdf?
    soft_frac = kernel(soft_ent[0])[0] / M # frac = density / M
    soft_frac = np.clip(soft_frac, 0, 1)
    across_mode_var = soft_frac < args.amv_thresh

    # does mode_ents' pdf peak near 0.0?
    max_ent = x_range[np.argmax(pdf)] # where is the max. of pdf?
    within_mode_var = max_ent > args.wmv_thresh

    return kernel, pdf, x_range, soft_frac, max_ent, within_mode_var, across_mode_var

def plot_mode_ents_pdf_per_roi(
        args, ax, roi, kernel, pdf, x_range, 
        mode_ents, soft_ent, soft_frac, max_ent, 
        within_mode_var, across_mode_var
    ):
    M = np.max(pdf)

    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    ax.axvline(0, ymin, ymax, lw=2, linestyle='-', color='tab:gray', alpha=0.75)
    ax.axhline(0, xmin, xmax, lw=2, linestyle='-', color='tab:gray', alpha=0.75)

    ax.plot(x_range, pdf, 'b-', lw=2, label='PDF')
    ax.scatter(mode_ents, np.full_like(mode_ents, -0.05), color='tab:grey', marker='x')

    ax.axvline(soft_ent, color='tab:red', lw=2, label='soft_ent')
    ax.scatter(soft_ent, [soft_frac], color='tab:red', marker='o', label=f'({soft_ent[0]:.2f},{soft_frac:.2f})')
    ax.axhline(args.amv_thresh, xmin, xmax, lw=2, linestyle='--', color='tab:red', alpha=0.75, label=f'')

    ax.scatter([max_ent], [1.0], color='tab:green', marker='o', label=f'({max_ent:.2f},{1.00:.2f})')
    # ax.scatter([0.0], [orig_frac], color='tab:blue', marker='o', label=f'({0.00:.2f},{orig_frac:.2f})')
    ax.axvline(max_ent, -0.05, 1.0, lw=2, linestyle='-', color='tab:green', alpha=1.0)
    ax.axvline(args.wmv_thresh, -0.05, 1.0, lw=2, linestyle='--', color='tab:green', alpha=0.75, label=f'')
    
    x=np.arange(0.0, max_ent, x_range[1]-x_range[0])
    ax.fill_between(
        x=x, 
        y1=np.zeros_like(x),
        y2=kernel(x) / M,
        color='tab:green',
        alpha=0.2,
    )
    
    ax.set_xlabel('Values')
    ax.set_ylabel('Density')
    ax.set_title(f'roi {roi:03d}: ({within_mode_var}, {across_mode_var})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    return None

def get_and_plot_mem_variability(
    args,       
):
    pass


In [10]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def var_types_to_nifti(args, X, folder, name=f'variability-type'):
    os.system(f'mkdir -p {folder}')
    in_files = []
    for idx_type in range(0, 4):
        x = X[:, idx_type]
        x_img = np.zeros_like(parcels)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels == roi) * (x[idx])
        
        file = f'{folder}/vartype-{idx_type:01d}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    out_file = f'{folder}/desc-{name}.nii.gz'
    concatenate(in_files, out_file)
    return None

In [11]:
def plot_ents_and_pis(args, mode_ents, soft_ent, pis, level=-1):
    nrows=1 
    ncols=1 + len(mode_ents)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4*ncols, 50*nrows), dpi=90)
    fig.tight_layout(h_pad=3, w_pad=3)
    
    ax = axs[0]
    X = mode_ents
    Y = np.expand_dims(soft_ent, axis=0)
    ax = plot_entropy_comparison(args, ax, X, Y, level)

    for col in range(1, ncols):
        ax = axs[col]
        sns.heatmap(pis[col-1], ax=ax, cmap=cc.cm.CET_L12)
    
    return fig

In [12]:
def entropy_comparisons(args, df, soft_ent, out_folder):
    # df = indiv_marginals_df
    mode_ents = np.stack(df['entropy'].to_list(), axis=0)
    mode_rank_ents = np.stack(df['rank_entropy'].to_list(), axis=0)
    pis = df['pi'].to_list()

    var_types = get_mem_variability_type(args, X=mode_ents, Y=soft_ent)
    fig = plot_ents_and_pis(args, mode_ents, soft_ent, pis)
    with open(f'{out_folder}/desc-variability-type.pkl', 'wb') as f:
        pickle.dump(var_types, f)
    fig.savefig(f'{out_folder}/desc-variability-type.pdf', bbox_inches='tight')
    return var_types, fig

def entropy_comparisons_all_animals(args, indiv_files, sf_files):
    for indiv_file, sf_file in tqdm(zip(indiv_files, sf_files)):
        args.sub = [s for s in indiv_file.split('/') if 'sub' in s][0].split('-')[-1]
        out_folder = f'{ESTIM_path}/individual/sub-{args.sub}/compare-entropies/{SBM}'
        os.system(f'mkdir -p {out_folder}')
        
        with open(indiv_file, 'rb') as f:
            indiv_marginals_df = pickle.load(f)
        
        indiv_marginals_df['pi'] = indiv_marginals_df['pi'].apply(lambda pi: np.round(pi, decimals=3))
        indiv_marginals_df[['entropy', 'rank_entropy']] = indiv_marginals_df['pi'].apply(lambda pi: pd.Series(mem_entropy(args, pi)))
        
        if args.sbm in ['a', 'd', 'o']:
            with open(sf_file, 'rb') as f:
                Grps_mat, soft_comms = pickle.load(f)
            soft_ent, soft_rank_ent = mem_entropy(args, soft_comms)
            
            var_types, fig = entropy_comparisons(args, indiv_marginals_df, soft_ent, out_folder)
            plt.close('all')
            var_types_to_nifti(args, var_types, out_folder)

        if args.sbm in ['h']:
            with open(sf_file, 'rb') as f:
                Grps_mats, soft_comms = pickle.load(f)
            
            for level, sft_cms in enumerate(soft_comms):
                df = indiv_marginals_df[indiv_marginals_df['level'] == level]
                os.system(f'mkdir -p {out_folder}/level-{level}')

                soft_ent, soft_rank_ent = mem_entropy(args, sft_cms)
                var_types, fig = entropy_comparisons(args, df, soft_ent, f'{out_folder}/level-{level}')
                plt.close('all')     
                var_types_to_nifti(args, var_types, f'{out_folder}/level-{level}')   
        # break
    return None

In [13]:
def analyze_group_var_types(matrix):
    n_samples = matrix.shape[-1]  # should be 10
    
    # probabilities
    probabilities = np.mean(matrix, axis=-1)
    
    # Get mode and its probability (most frequent column/var_type); may have multiple
    modes, mode_probs = list(zip(*[(list(np.where(row == np.max(row))[0]), np.max(row)) for row in probabilities]))
    modes = MultiLabelBinarizer(classes=list(range(4))).fit_transform(modes)

    # Calculate entropy to measure uncertainty
    # High entropy means more uncertainty in the distribution
    entropy = stats.entropy(probabilities, base=2, axis=-1) / np.log2(probabilities.shape[-1])

    # min value in probabilities
    probabilities = np.where(probabilities == 0.0, 0.01, probabilities)
    
    return {
        'modes': modes,
        'mode_probabilities': mode_probs,
        'entropy': entropy,
        'full_distribution': probabilities,
    }

def get_and_plot_group_stats(args, dfs, level=-1):
    X = np.stack(dfs['var_types'].to_list(), axis=-1)
    group_var_stats = analyze_group_var_types(X)
    if level == -1:
        out_folder = f'{ESTIM_path}/group/compare-entropies/{SBM}'
    else:
        out_folder = f'{ESTIM_path}/group/compare-entropies/{SBM}/level-{level}'
    os.system(f'mkdir -p {out_folder}')
    var_types_to_nifti(args, X=group_var_stats['full_distribution'], folder=f'{out_folder}', name=f'variability-type-full-dist')
    var_types_to_nifti(args, X=group_var_stats['modes'], folder=f'{out_folder}', name=f'variability-type-modes')
    np.savetxt(f'{out_folder}/desc-variability-type-full-dist.txt', group_var_stats['full_distribution'], fmt='%.2f')
    np.savetxt(f'{out_folder}/desc-variability-type-modes.txt', group_var_stats['modes'], fmt='%.2f')
    return group_var_stats

def group_mem_variability(args):
    if args.sbm in ['a', 'd', 'o']:
        dfs = []
        var_type_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/compare-entropies/{SBM}/desc-variability-type.pkl'))
        for file in var_type_files:
            with open(file, 'rb') as f:
                var_types = pickle.load(f)
            df = pd.DataFrame(dict(
                sub=[s for s in file.split('/') if 'sub' in s][0].split('-')[-1],
                var_types=[var_types],
            ))
            dfs += [df]
        dfs = pd.concat(dfs).reset_index(drop=True)
        group_var_stats = get_and_plot_group_stats(args, dfs, level=-1)

    if args.sbm in ['h']:
        dfs = []
        var_type_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/compare-entropies/{SBM}/level-*/desc-variability-type.pkl'))
        for file in var_type_files:
            with open(file, 'rb') as f:
                var_types = pickle.load(f)
            df = pd.DataFrame(dict(
                sub=[s for s in file.split('/') if 'sub' in s][0].split('-')[-1],
                level=int([l for l in file.split('/') if 'level' in l][0].split('-')[-1]),
                var_types=[var_types],
            ))
            dfs += [df]
        dfs = pd.concat(dfs).reset_index(drop=True)

        group_var_stats = []
        for level in dfs['level'].unique():
            group_var_stats += [get_and_plot_group_stats(args, dfs[dfs['level'] == level], level=level)]
    
    return group_var_stats

In [14]:
sbms_list = [
    (True, 'a'),
    (True, 'd'),
    (True, 'h'),
    (False, 'd'),
    (False, 'h'),
]

In [15]:
for sbm in sbms_list[:]:
    args.dc, args.sbm = sbm

    args.nested = args.sbm == 'h'

    args.force_niter = 40000
    args.num_draws = int((1/2) * args.force_niter)

    def sbm_name(args):
        dc = f'dc' if args.dc else f'nd'
        dc = f'' if args.sbm in ['a'] else dc
        file = f'sbm-{dc}-{args.sbm}'
        return file

    SBM = sbm_name(args)
    print(SBM)

    indiv_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-marginals-df.pkl', recursive=True))
    display(indiv_files)
    
    sf_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/soft-communities/{SBM}/desc-matrices.pkl'))
    display(sf_files)

    args.stats_column = 'entropy'
    entropy_comparisons_all_animals(args, indiv_files, sf_files)
    group_var_stats = group_mem_variability(args)

sbm--a


['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/gr

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC01/soft-communities/sbm--a/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC02/soft-communities/sbm--a/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC03/soft-communities/sbm--a/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-posit

241120-21:23:24,483 nipype.interface INFO:
	 stderr 2024-11-20T21:23:24.483590:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241120-21:23:24,487 nipype.interface INFO:
	 stderr 2024-11-20T21:23:24.486996:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241120-21:23:24,489 nipype.interface INFO:
	 stderr 2024-11-20T21:23:24.489429:*+ WARNING: Set TR of output dataset to 1.0 s
241120-21:23:24,560 nipype.interface INFO:
	 stderr 2024-11-20T21:23:24.560637:++ elapsed time = 0.1 s
241120-21:23:25,46 nipype.interface INFO:
	 stderr 2024-11-20T21:23:25.046646:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241120-21:23:25,49 nipype.interface INFO:
	 stderr 2024-11-20T21:23:25.049578:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241120-21:23:25,51 nipype.interface INFO:
	 stderr 2024-11-20T21:23:25.051637:*+ WARNING: Set TR of output dataset to 1.0 s
241120-21:23:25,102 nipype.interface INFO:
	 stderr 2024-11-20

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm-dc-d/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm-dc-d/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm-dc-d/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-

...

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC01/soft-communities/sbm-nd-d/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC02/soft-communities/sbm-nd-d/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC03/soft-communities/sbm-nd-d/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold

241120-21:23:30,886 nipype.interface INFO:
	 stderr 2024-11-20T21:23:30.886003:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241120-21:23:30,889 nipype.interface INFO:
	 stderr 2024-11-20T21:23:30.889374:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241120-21:23:30,891 nipype.interface INFO:
	 stderr 2024-11-20T21:23:30.891876:*+ WARNING: Set TR of output dataset to 1.0 s
241120-21:23:30,957 nipype.interface INFO:
	 stderr 2024-11-20T21:23:30.957215:++ elapsed time = 0.1 s
241120-21:23:31,439 nipype.interface INFO:
	 stderr 2024-11-20T21:23:31.439706:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241120-21:23:31,443 nipype.interface INFO:
	 stderr 2024-11-20T21:23:31.443105:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241120-21:23:31,445 nipype.interface INFO:
	 stderr 2024-11-20T21:23:31.445523:*+ WARNING: Set TR of output dataset to 1.0 s
241120-21:23:31,507 nipype.interface INFO:
	 stderr 2024-11

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm-nd-h/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm-nd-h/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm-nd-h/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-

['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC01/soft-communities/sbm-nd-h/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC02/soft-communities/sbm-nd-h/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-20/layer-individual/unit-ses/estimates/individual/sub-SLC03/soft-communities/sbm-nd-h/desc-matrices.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold

241120-21:23:31,977 nipype.interface INFO:
	 stderr 2024-11-20T21:23:31.976978:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241120-21:23:31,980 nipype.interface INFO:
	 stderr 2024-11-20T21:23:31.980463:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241120-21:23:31,982 nipype.interface INFO:
	 stderr 2024-11-20T21:23:31.982953:*+ WARNING: Set TR of output dataset to 1.0 s
241120-21:23:32,42 nipype.interface INFO:
	 stderr 2024-11-20T21:23:32.042917:++ elapsed time = 0.1 s
241120-21:23:32,531 nipype.interface INFO:
	 stderr 2024-11-20T21:23:32.531076:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
241120-21:23:32,534 nipype.interface INFO:
	 stderr 2024-11-20T21:23:32.534647:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
241120-21:23:32,537 nipype.interface INFO:
	 stderr 2024-11-20T21:23:32.537178:*+ WARNING: Set TR of output dataset to 1.0 s
241120-21:23:32,588 nipype.interface INFO:
	 stderr 2024-11-